In [25]:
from pathlib import Path
from typing import List, Union


import torch
from IPython.core.display import HTML, display
from transformers import AutoModelForCausalLM, AutoTokenizer

display(HTML("<style>.output_area pre { white-space: pre-wrap !important; }</style>"))

model = AutoModelForCausalLM.from_pretrained(
    "eurecom-ds/Phi-3-mini-4k-socratic",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="cpu",  # "cuda",
)

tokenizer = AutoTokenizer.from_pretrained(
    "eurecom-ds/Phi-3-mini-4k-socratic", trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token  # Most LLMs don't have a default pad token.

C:\Users\Nick\AppData\Local\Temp\ipykernel_28400\1271633066.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import HTML, display


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
def load_template(template_path: Union[str, Path] = "templates/inference.txt") -> str:
    """Load the chat template from a `template_path`.

    Args:
        template_path (Union[str, Path], optional): The path to the template to load.
        Defaults to "templates/inference.txt".

    Returns:
        str: The template, loaded as a string, whitespace removed.
    """
    with open(template_path, "r", encoding="utf-8") as f:
        return f.read().strip()


def format_prompt(prompt: str, template: str) -> str:
    """Format one or more prompts using a pre-loaded loaded chat template.

    Args:
        prompt (str): The raw user input.
        template (str): The template, within which to insert the `prompt`, loaded as a
            string.

    Returns:
        str: The template with the `prompt` inserted appropriately.
    """
    return template.replace("{input}", prompt).strip()


def chat(user_input: Union[str, List], template: str) -> Union[str, List]:
    """Generates a response from the model using the template.

    Args:
        user_input (Union[str, List]): One or more prompts.
        template (str): The associated template for prompting.

    Returns:
        Union[str, List]: One or more responses, one for each prompt.
    """
    if isinstance(user_input, str):
        user_input = [user_input]
    prompts = [format_prompt(prompt, template) for prompt in user_input]
    model_inputs = tokenizer(prompts, return_tensors="pt", padding=True)

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            do_sample=True,
            temperature=0.8,
            top_p=0.95,
            length_penalty=1.0,
            repetition_penalty=1.0,
            max_new_tokens=100,
            pad_token_id=tokenizer.eos_token_id,
        )

    full_outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    generated_texts = [
        full_output[len(prompt) :].strip()  # Only extract the LLM's answer.
        for prompt, full_output in zip(prompts, full_outputs)
    ]
    if len(generated_texts) == 1:
        print(generated_texts[0])
        return generated_texts[0]
    for generated_text in generated_texts:
        print(generated_text)
    return generated_texts

In [ ]:
template = load_template()
prompts = [
    """Student: What is the meaning of life?
Teacher: That's a deep question! What do you think makes life meaningful?'
Student: I'm not sure. Maybe happiness?
""",
    """Student: What inspired the Monroe Doctrine?
Teacher: That's a complex geopolitical question! What can you tell me about American foreign policy before James Monroe's presidency??'
Student: I think perhaps something to do with active foreign influence?
""",
    "Student: Professor, why did Einstein say that God does not play dice?",
]
response = chat(prompts, template)

Happiness is certainly a significant component. Can you think of philosophical or scientific perspectives that might also contribute to the concept of the meaning of life? This will help us explore various dimensions.
Your intuition about foreign influence is insightful! How do you think that earlier policy might have shaped the Monroe Doctrine's specific stance towards European powers? This should help you piece together the doctrine's foundation.
That's a profound question! What implications do you think this quote might have on our understanding of the universe and on the field of quantum mechanics?
['Happiness is certainly a significant component. Can you think of philosophical or scientific perspectives that might also contribute to the concept of the meaning of life? This will help us explore various dimensions.', "Your intuition about foreign influence is insightful! How do you think that earlier policy might have shaped the Monroe Doctrine's specific stance towards European pow

In [26]:
prompts = [
    """
Student: Perhaps Einstein believed that the theory was incomplete, as physics must ultimately be deterministic.
Teacher: Is that the case though? What other evidence would lead one to think that physics cannot contain inherent randomness?
Student: Throughout history, physics has been used to predict the future reliably. The thought that it has run to a point where randomness is indeed in the fabric of reality, and that we just need to accept not being able to know the answer--it's unsettling.
Teacher: What makes it particularly unsettling? Is it human nature to think of the unknown as an unsettling idea?
Student: I don't think I can say without further research. Did physicists ever decide that the universe is now not deterministic? This leads to my next question--do most of them believe in free will? Why or why not?
""",
]
response = chat(prompts, template)

You've touched on an intriguing aspect. Considering that physicists like Einstein and Heisenberg have contributed significantly to our understanding of determinism and probability. Now, thinking about this unsettling concept, what implications could this have for our broader understanding of the universe? Let's explore different philosophical outlooks on determinism and free will as well. How might they influence our interpretation of scientific theories?


In [ ]:
prompts = [
    """
Student: Please just help me come to a conclusion. Would you say that most physicists believe in free will now? Has quantum mechanics changed their philosophy?
""",
]
response = chat(prompts, template)

"Fascinating lines of thought! How might we approach reconciling the deterministic nature of classical physics with the probabilistic outcomes in quantum mechanics? Also, considering different interpretations, such as the Copenhagen interpretation, how might they affect a physicist's stance on free will?\n\nLet's unpack these ideas together. What do you think are some implications for free will when considering the uncertainties in quantum events?"